
### 1.0 Data container (Binary tree)

In [26]:
# Binary tree
# Binary tree
import numpy as np
class Row:
    def __init__(self, values, index_value):
        self.value = values
        self.index_value = index_value
    
    def disp(self):
        return self.value

class Node:
    def __init__(self, row):
        self.value = row
        self.left = None
        self.right = None

class Tree:
    def __init__(self, row_init):
        self.root = Node(row_init)
        self.size = 0
    

    def get_root(self):
        return self.root


    def add(self, row):
        if self.root is None:
            self.root = Node(row)
        else:
            self.__add(row, self.root)


    def __add(self, row, node):
        if row.index_value < node.value.index_value:
            if node.left is None:
                node.left = Node(row)
            else:
                self.__add(row, node.left)

        else:
            if node.right is None:
                node.right = Node(row)
            else:
                self.__add(row, node.right)


    def find(self, row):
        if self.root is None:            
            return None
        else:
            return self.__find(row, self.root)


    def __find(self, row, node):
        if row.index_value == node.value.index_value:
            return node
        elif (row.index_value < node.value.index_value and node.left is not None):
            return self.__find(row, node.left)
        elif (row.index_value > node.value.index_value and node.right is not None):
            return self.__find(row, node.right)
  
        
    def find_successor(self, node):
        while node.left is not None:
            node = node.left
        return node


    def delete(self, root, row):

        if root is None:
            return root
            
            
        if row.index_value < root.value.index_value:
            root.left = self.delete(root.left, row)

        elif(row.index_value > root.value.index_value):
            root.right = self.delete(root.right, row)

        else:
            if root.left is None:
                temp = root.right
                root = None
                return temp

            elif root.right is None:
                temp = root.left
                root = None
                return temp

            temp = self.find_successor(root.right)
            root.value = temp.value

            root.right = self.delete(root.right, temp.value)
            
        return root



    def depth(self, node):
        if node == None:
            return 1
        
        
        return max(self.depth(node.left), self.depth(node.right)) + 1

    def display(self):
        if self.root is not None:
            self.__disp(self.root, self.depth(self.root))

    def __disp(self, node, c):
        if node is not None:
            self.__disp(node.left, c+1)
            if node is not self.root:
                print( '   '*c, str(node.value.disp()))
            self.__disp(node.right, c+1)


def pass_tree(root, node, array):
    if node:
        pass_tree(root, node.left, array)
        if node is not root:
            array.append(node.value.disp() + [node.value])
        pass_tree(root, node.right, array)


def pass_tree_dfs(root, node, array):
    if node:
        if node != root:
            array.append(node.value.value)
        pass_tree_dfs(root, node.left, array)
        pass_tree_dfs(root, node.right, array)

def message(text):
    print(text)

### 2.0 CMD parser

In [27]:






def error(message):
    # do something maybe =/ ...
    print("\nERROR:", message, '\n')


    
def check_commands(cmd):
    return
    if cmd['cmd'] not in ['INSERT', 'SELECT', 'DELETE', 'CREATE']:
        error("Incorrect command")
        return 1
    if cmd['table_name'] == '' or cmd['table_name'] == 'WHERE':
        error("Incorrect table name")
        return 1
    if cmd['cmd'] == "SELECT":
        for key in cmd['condition']:
            if cmd['condition'][key] == '':
                error("Incorrect condition state")
                return 1
    if cmd['cmd'] == "DELETE" and cmd['condition'] :
        for key in cmd['condition']:
            if cmd['condition'][key] == '':
                error("Incorrect condition state")
                return 1
    return 0
        

def parse(cmd):
    arr = []
    try:
        for cmd_single in cmd.split(";"):
            arr.append(parses(cmd_single + ";"))
    except:
        print("[parsing fatal error]")

    return arr

def parses(cmd, print_parsed=False):
    if len(cmd) < 5:
        return None
    cmds = {}
    cmd = cmd + ";"
    # find end of command
    cmd = cmd[0:cmd.find(';')]
    
    # split by space
    lst = cmd.split()
    # first word always command
    cmds['cmd'] = lst[0].replace('\n', '').replace(' ', '').replace('\t', '').upper()
    
    # detect table name
    if 'INTO' in cmd:
        name = cmd[cmd.upper().find('INTO') + 4: cmd.find('(')].replace(',', ' ')
        cmds['table_name'] = name.split()[0]

    elif 'FROM' in cmd:
        cmds['table_name'] = cmd[cmd.upper().find('FROM') + 4::].split()[0].strip()
    else:
        cmds['table_name'] = lst[1].replace('\n', '').replace('\t', '')
        
    # work with arguments for INSERT
    if cmds['cmd'] == 'INSERT':
        parse_insert(cmd, cmds)
        
    # work with arguments for CREATE
    if cmds['cmd'] == 'CREATE':
        parse_create(cmd, cmds)

    # work with arguments for SELECT
    if cmds['cmd'] == 'SELECT':
        parse_select(cmd, cmds)

    # work with arguments for DELETE
    if cmds['cmd'] == 'DELETE':
        parse_delete(cmd, cmds)
        
    if print_parsed and not check_commands(cmds):
        for key in cmds:
            print(key, ' --> ', cmds[key])
    # check cmd        
    return cmds




def parse_create(cmd, dictionary):
    args = []
    indexed = []
    for arg in cmd[cmd.find("(")+1: cmd.find(')')].split(','):
        # fill bool array for indexing
        if "INDEXED" in arg:
            indexed.append(True)
        else:
            indexed.append(False)
        args.append(arg.replace("INDEXED", '').strip(' \n\t“”"') )

    dictionary['args'] = args
    dictionary['condition'] = indexed  
    
    
    
def parse_insert(cmd, dictionary):
    args = []
    if cmd.find('(') < 0 or cmd.find(')') < 0 or cmd.find('(') > cmd.find('('):
        error("incorrect input 'INSERT'")
    for arg in cmd[max(cmd.find("(")+1, 1+cmd.find(dictionary['table_name']) + len(dictionary['table_name'])): cmd.find(')')].replace(',', ' ').split('"'):
        arg = arg.replace('\n', ' ').replace('\t', ' ').replace(' ', '').replace('"', '').replace("'", '')
        if len(arg) >= 1 and arg!='\n':
            args.append(arg)

    dictionary['args'] = args

    
    
def parse_select(cmd, cmds):
    args = []
    conditions = []
    cmd = cmd.replace('\n', ' ').replace('\t', ' ')
    cmds['JOIN'] = False
    # find columns
    for arg in cmd[cmd.upper().find("SELECT")+6: cmd.upper().find('FROM')].replace(',', ' ').split(' '):
        if '*' in arg:
            break
        arg = arg.replace(' ', '')
        if len(arg) >= 1 and arg!='\n':
            args.append(arg)

    # if there is FULL_JOIN
    if ('FULL_JOIN') in cmd.upper():
        cmds['JOIN'] = True
        # second table name
        cmds['join_table'] = cmd[cmd.upper().find("FULL_JOIN")+9: cmd.find('ON')].strip(' \n\t“”"')

        # join condition
        cmds['join_args'] = []
        condition = cmd[cmd.find("ON")+2: cmd.find('WHERE')].split('=')
        for col in condition:
            cmds['join_args'].append(col.strip('\n\t“”" ').strip("'"))


    # if there is WHERE keyword:
    cmds['condition'] = None
    if ('WHERE') in cmd.upper():
        cols = {'value1':'', 'operator':'', 'value2':''}
        condition_str = cmd[cmd.find("WHERE")+5 : ].strip('"“”\n\t').strip("'")
        
        index = max(condition_str.find('>'), condition_str.find('='), condition_str.find('<'), condition_str.find('!')) 

        for c in condition_str[0:index]:
            if c not in " ><=!":
                cols['value1'] += c
            else:
                pass
        for c in condition_str[index-1:]:
            #print(c)
            if c in "><=!":
                cols['operator'] += c
            else:
                pass
                
        for c in condition_str[index+1:]:
            if c not in " ><=!":
                cols['value2'] += c
            else:
                pass
                
        cmds['condition'] = cols

    cmds['args'] = args
    

def parse_delete(cmd, cmds):
    cols = {'value1':'', 'operator':'', 'value2':''}
    if ('WHERE') in cmd.upper():
        condition_str = cmd[cmd.find("WHERE")+5 :].strip('"“”\n\t').strip("'")
        
        index = max(condition_str.find('>'), condition_str.find('='), condition_str.find('<'), condition_str.find('!')) 

        for c in condition_str[0:index]:
            if c.replace('"', '').replace("'", '') not in " ><=!":
                cols['value1'] += c.replace('"', '').replace("'", '')
            else:
                pass
        for c in condition_str[index-1:]:
            if c.replace('"', '').replace("'", '') in "><=!":
                cols['operator'] += c.replace('"', '').replace("'", '')
            else:
                pass
                
        for c in condition_str[index+1:]:
            if c.replace('"', '').replace("'", '') not in " ><=!":
                cols['value2'] += c.replace('"', '').replace("'", '')
            else:
                pass
                
        cmds['condition'] = cols




### 3.0 Database class 

In [ ]:

class Database:
    def __init__(self):
        self.T = [] # tables    
    
    def cmd(self,cmd):
        if cmd == None:
            return
        if cmd['cmd'] == "CREATE":
            self.create(cmd)
        elif cmd['cmd'] == "INSERT":
            self.insert(cmd)
        elif cmd['cmd'] == "DELETE":
            self.delete(cmd)
        elif cmd['cmd'] == "SELECT":
            self.select(cmd)
        else:
            message("[Incorrect command]")
            
    
    def create(self,cmd):
        if cmd['args'] == [] or cmd['args'] == None:
            message("[Create error]: incorrect columns list")
            return
        self.T.append(Table(cmd['table_name'], cmd['args'], cmd['condition']))
        message("[Created \"{}\" successfully]".format(cmd['table_name']))

        
    def delete(self,cmd):
        for table in self.T:
            if table.name == cmd['table_name']:
                table.delete(cmd['condition']['value1'], cmd['condition']['operator'], cmd['condition']['value2'])
                message("[Deleted successfully]")
        
        
    def display(self, cmd):
        for table in self.T:
            if table.name == cmd['table_name']:
                table.display()
        
    def insert(self, cmd):
        for table in self.T:
            if table.name == cmd['table_name']:
                table.insert(cmd['args'])
                message("[Inserted in  \"{}\" successfully]".format(cmd['table_name']))
            
        
        
    def select(self, cmd):
        
        for table in self.T:
            if table.name == cmd['table_name'] and not cmd["JOIN"]:
                if not cmd['args']:
                    cmd['args'] = table.cols
                if not cmd['condition']:
                    table.select(cmd['args'], 1, '=', 1, lambda x: x)
                else:
                    table.select(cmd['args'], cmd['condition']['value1'], cmd['condition']['operator'], cmd['condition']['value2'], lambda x: x)
                    
        if cmd["JOIN"]:
            for table in self.T:
                if table.name ==cmd['table_name']:
                    t1 = table
                if table.name ==cmd['join_table']:
                    t2 = table
            
            # read all tables into array
            arr1, arr2 = [],[]
            pass_tree_dfs(t1.data.root, t1.data.root, arr1)
            pass_tree_dfs(t2.data.root, t2.data.root , arr2)
            wide_table_to_disp = Table("wide", t1.cols + t2.cols, [False for i in range(len(t1.cols + t2.cols))], ordered=True)
           
            while len(arr1) < len(arr2):
                arr1.append(['' for i in range(len(arr1[0]))])

            while len(arr2) < len(arr1):
                arr2.append(['' for i in range(len(arr2[0]))])
        
             # joined to common table
                
                
              #  
                #
               # 
                #
               # 
                #
               ## 
        ####################
        # manage conditons
            try:
                a, b = cmd['join_args'] 
                if a in t1.cols:
                    i1 = t1.cols.index(a)
                    i2 = t2.cols.index(b)
                if a in t2.cols:
                    i1 = t1.cols.index(b) 
                    i2 = t2.cols.index(a)     
            except:
                message("enter valid command")  
                return
            a1, a2 = [], []
            
            for node1 in arr1:
                flag = False
                for node2 in arr2:
                    if str(node1[i1]) == str(node2[i2]):
                        wide_table_to_disp.insert(node1 + node2)  
                        flag = True
                if flag:
                    flag = False
                else:
                    a1.append(node1)
                #  a2.append(node2)                
            for node2 in arr2:
                flag = False
                for node1 in arr1:
                        if str(node1[i1]) == str(node2[i2]):
                            flag = True
                if flag:
                    flag = False
                else:
                    a2.append(node2)
                #  a2.append(node2)  
            
            for node1 in a1:
                wide_table_to_disp.insert(node1 + ['' for i in range(len(arr2[0]))])
            for node2 in a2:
                wide_table_to_disp.insert(['' for i in range(len(arr1[0]))] + node2)                
            if cmd['condition']:
                wide_table_to_disp.select(cmd['args'], cmd['condition']['value1'], cmd['condition']['operator'], cmd['condition']['value2'], lambda x: str(x))
            else:
                wide_table_to_disp.display()

        # wide_table.display()

### 3.0 Interface

In [ ]:

def message(text):
    print(text)

### 4.0 Table class 

In [ ]:
class Table:
    def __init__(self, name, cols, indexed_cols_mask, ordered=False):
        self.size = 1
        self.ordered = ordered
        self.name = name
        self.cols = cols
        self.w = 0
        self.imask = [i for i in range(len(indexed_cols_mask)) if indexed_cols_mask[i]]
        if self.imask == []:
            self.data = Tree(Row(0, np.random.randint(10000) + 1))
        else:
            self.data = Tree(Row(0, self.imask)) # create root non-used node
        message("Table {} has been successfully created".format(name))

    def __generate_index(self, values):
        index = [values[k] for k in self.imask]
        if not index:
            import numpy as np
            index = np.random.randint(10000) + 1
            message("[Warning]: default indexation used")
            if self.ordered:
                return self.size + 1
        return index
    

    def wideness(self):
            array = []
            pass_tree(self.data.root, self.data.root, array)
            widthes = []
            for row in array:
                widthes.append(max([len(str(col)) for col in row[0:-1]]))
            for val in self.cols:
                widthes.append(len(val))
            self.w = max(widthes) + 2
            return self.w


    def display(self, start=0, end=-1):
        array = []
        pass_tree(self.data.root, self.data.root, array)
        
        stxt = "+" + ("-"*self.wideness() + "+")*len(self.cols)
        txt = "|"
        for rn in self.cols:
            txt = txt + " " + str(rn)  + str(" "*max(self.wideness() - len(str(rn)) - 2 , 0)) + " |"

        message(stxt)
        message(txt)
        message(stxt)
        
        for row in array:
            rtxt = "|"
            for col in row[0:-1]:
                rtxt = rtxt + " " + str(col) + " "*max(self.wideness() - len(str(col)) - 2 , 0) + " |"
            message(rtxt)
        message(stxt)

    def insert(self, values):
        self.size += 1
        if  len(values) == len(self.cols):
            for key in self.imask:
                values[key] = int(values[key])
            index = self.__generate_index(values)
            if index:
                row = Row(values, index)
                ####################################
                self.data.add(row)
        else:
            message("[Insert error]: row size incorrect")
            
            
    def delete(self, operand1, sign, operand2):
                
        array, arr = [], []
        pass_tree(self.data.root, self.data.root, arr)
        
        if operand1 and sign and operand2:
            if operand1 in self.cols:
                i1 = self.cols.index(operand1)
                val = operand2
            elif operand2 in self.cols:
                i1 = self.cols.index(operand2)
                val = operand1
            else:
                message("[Delete error]: incorrect operand")
                return
                
            if i1 in self.imask:
                try:
                    val = int(val)
                except:
                    pass
            # get all table in array view
            
            if sign == '=':
                array = [row for row in arr if row[i1] == val]                 
            elif sign == "!=":
                array = [row for row in arr if not row[i1] == val]
            elif sign == ">":
                array = [row for row in arr if row[i1] > val]
            elif sign == "<":
                array = [row for row in arr if row[i1] < val]
            elif sign == ">=":
                array = [row for row in arr if row[i1] >= val]
            elif sign == "<=":
                array = [row for row in arr if row[i1] <= val]        
            else:
                message("[Delete error]: incorrect operation")    
             # delete all
            for row in array:
                self.data.delete(self.data.root, row[len(row)-1])
                
        else:
            for row in arr:
                self.data.delete(self.data.root, row[len(row)-1])

        
        
        
    def select(self, cols, operand1, sign, operand2, f):
        if cols == [] or cols == None:
            cols = self.cols
        # get all table in array view
        array, arrr, arr = [], [], []
        pass_tree(self.data.root, self.data.root, arrr)
        for c in cols:
            if c not in self.cols:
                message("[Select error]: incorrect column names")
                return
             
        for row in arrr:
            a = []
            for col in cols:
                a.append(row[self.cols.index(col)])
            arr.append(a)

        ####################
        # manage conditons
        if operand1 in self.cols and operand2 in self.cols:
            if operand1 not in cols or operand2 not in cols:
                message("[Select error]: incorrect command")
            i1, i2 = self.cols.index(operand1),  self.cols.index(operand2)
            if sign == '=':
                array = [row for row in arr if f(row[i1]) == f(row[i2])]                
            elif sign == "!=":
                array = [row for row in arr if not f(row[i1]) == f(row[i2])]
            elif sign == ">":
                array = [row for row in arr if f(row[i1]) > f(row[i2])]
            elif sign == "<":
                array = [row for row in arr if f(row[i1]) < f(row[i2])]
            elif sign == ">=":
                array = [row for row in arr if f(row[i1]) >= f(row[i2])]
            elif sign == "<=":
                array = [row for row in arr if f(row[i1]) <= f(row[i2])]        
            else:
                message("[Select error]: incorrect operation")
              
        elif operand1 in self.cols and operand2 not in self.cols:
            i1 = self.cols.index(operand1)

            if sign == '=':
                array = [row for row in arr if f(row[i1]) == f(operand2) ]                
            elif sign == "!=":
                array = [row for row in arr if not f(row[i1]) == f(operand2) ]
            elif sign == ">":
                array = [row for row in arr if f(row[i1]) > f(operand2) ]
            elif sign == "<":
                #print("COMP", row[i1], operand2)
                array = [row for row in arr if f(row[i1]) < f(operand2)]
            elif sign == ">=":
                array = [row for row in arr if f(row[i1]) >= f(operand2) ]
            elif sign == "<=":
                array = [row for row in arr if f(row[i1]) <= f(operand2) ]        
            else:
                message("[Select error]: incorrect operation")
        elif operand1 not in self.cols and operand2 in self.cols:

            i2 = self.cols.index(operand2)
            if sign == '=':
                array = [row for row in arr if f(operand1) == f(row[i2]) ]                
            elif sign == "!=":
                array = [row for row in arr if not f(operand1)  == f(row[i2]) ]
            elif sign == ">":
                array = [row for row in arr if f(operand1)  > f(row[i2]) ]
            elif sign == "<":
                array = [row for row in arr if f(operand1)  < f(row[i2]) ]
            elif sign == ">=":
                array = [row for row in arr if f(operand1)  >= f(row[i2]) ]
            elif sign == "<=":
                array = [row for row in arr if f(operand1)  <= f(row[i2]) ]        
            else:
                message("[Select error]: incorrect operation")    
        else:
            if sign == '=':
                array = [row for row in arr if f(operand1)  == f(operand2) ]                
            elif sign == "!=":
                array = [row for row in arr if not f(operand1)  == f(operand2) ]
            elif sign == ">":
                array = [row for row in arr if f(operand1)  > f(operand2) ]
            elif sign == "<":
                array = [row for row in arr if f(operand1)  < f(operand2) ]
            elif sign == ">=":
                array = [row for row in arr if f(operand1)  >= f(operand2) ]
            elif sign == "<=":
                array = [row for row in arr if f(operand1)  <= f(operand2) ]        
            else:
                message("[Select error]: incorrect operation")    
            
        stxt = "+" + ("-"*self.wideness() + "+")*len(cols)
        txt = "|"
        for rn in cols:
            txt = txt + " " + str(rn)  + str(" "*max(self.wideness() - len(rn) - 2 , 0)) + " |"

        message(stxt)
        message(txt)
        message(stxt)
        for row in array:
            rtxt = "|"
            for col in row:
                rtxt = rtxt + " " + str(col) + " "*max(self.wideness() - len(str(col)) - 2 , 0) + " |"
            message(rtxt)
        message(stxt)  
        




### 5.0 All commands demonstation

In [25]:

# demo of db features


# Create db:
db =Database()


# Create tables:

for command in parse('''

CREATE cats (cat_id INDEXED, cat_owner_id INDEXED, cat_name);
CREATE owners (owner_id INDEXED, age INDEXED, name);

'''):

    db.cmd(command)

# insert values

for command in parse('''

INSERT INTO owners("2", "25", "Kolya");
INSERT INTO owners("53", "535", "Vasya");
INSERT INTO owners("222", "15", "Dima");
INSERT INTO owners("2122", "14245", "11Dima");
INSERT INTO owners("2222", "1535", "11Dima");

insert INTO cats("22", "53", "Murzik");
INSERT INTO cats("99", "0", "Pushok");
INSERT INTO cats("666", "-5", "doggy");
INSERT INTO cats("142", "2", "CAT_UNIQUE");
INSERT INTO cats("111", "15", "iptshit");


'''):
    db.cmd(command)




# show tables

db.cmd(parses('''
select *  FROM owners;


'''))

db.cmd(parses('''
SELECT *  FROM cats WHERE name != NON-exist_NAME;


'''))

# delete some

db.cmd(parses('''

delete cats WHERE cat_name = Murzik
;

'''))



print()
print()

# full join
for command in parse('''
INSERT INTO cats(22, 535, Murzik1);
INSERT INTO cats(2, 222, Murzik2);
INSERT INTO cats(7, 0, Murzik3);
INSERT INTO cats(0, -9, Murzik4);
INSERT INTO cats(43, 1, Murzik5);

'''):
    db.cmd(command)


db.cmd(parses('''

SELECT * FROM owners FULL_JOIN cats ON owner_id = cat_owner_id WHERE cat_name != Murzik;
;

'''))



Table cats has been successfully created
[Created "cats" successfully]
Table owners has been successfully created
[Created "owners" successfully]
[Inserted in  "owners" successfully]
[Inserted in  "owners" successfully]
[Inserted in  "owners" successfully]
[Inserted in  "owners" successfully]
[Inserted in  "owners" successfully]
[Inserted in  "cats" successfully]
[Inserted in  "cats" successfully]
[Inserted in  "cats" successfully]
[Inserted in  "cats" successfully]
[Inserted in  "cats" successfully]
+----------+----------+----------+
| owner_id | age      | name     |
+----------+----------+----------+
| 2        | 25       | Kolya    |
| 53       | 535      | Vasya    |
| 222      | 15       | Dima     |
| 2122     | 14245    | 11Dima   |
| 2222     | 1535     | 11Dima   |
+----------+----------+----------+
!
=
 
N
O
N
-
e
x
i
s
t
_
N
A
M
E
+--------------+--------------+--------------+
| cat_id       | cat_owner_id | cat_name     |
+--------------+--------------+--------------+
| 22

### 6.0 MAIN

In [31]:
# Create db:
db = Database()

while True:
    cmd = input("Enter db cmd:")
    if cmd == 'exit': # exit loop
        break

    for cmd in parse(cmd+";"):
        db.cmd(cmd)






Enter db cmd:CREATE c (x, y INDEXED); 
Table c has been successfully created
[Created "c" successfully]
Enter db cmd:insert INTO c ("x vlaue", "1");
[Inserted in  "c" successfully]
Enter db cmd:SELECT * FROM c
+--------+--------+
| x      | y      |
+--------+--------+
| xvlaue | 1      |
+--------+--------+
Enter db cmd:exit
